<a href="https://colab.research.google.com/github/hasannader/Superstore-Sales/blob/main/Superstore_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [11]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
from datetime import datetime

# Load and preprocess dataset
df = pd.read_csv('train.csv')

# Data Cleaning
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%d/%m/%Y')
df['Year'] = df['Order Date'].dt.year
df['Month'] = df['Order Date'].dt.month_name()
df['Day of Week'] = df['Order Date'].dt.day_name()

# Simulate missing columns
np.random.seed(42)
sales_reps = ['Rep_' + str(i) for i in range(1, 11)]
df['Sales_Rep_ID'] = np.random.choice(sales_reps, size=len(df))
time_slots = ['Morning', 'Afternoon', 'Evening', 'Night']
df['Time of Day'] = np.random.choice(time_slots, size=len(df))
df['Profit'] = df['Sales'] * 0.2
df['Return_Flag'] = np.random.choice([0, 1], size=len(df), p=[0.95, 0.05])
df['Quantity_Sold'] = 1

# Calculate KPIs
monthly_sales = df.groupby([df['Order Date'].dt.to_period('M')])['Sales'].sum().reset_index()
monthly_sales['MoM Growth'] = monthly_sales['Sales'].pct_change() * 100
yearly_sales = df.groupby('Year')['Sales'].sum().reset_index()
yearly_sales['YoY Growth'] = yearly_sales['Sales'].pct_change() * 100
avg_order_size = df['Sales'].sum() / df['Quantity_Sold'].sum()
return_rate = (df['Return_Flag'].sum() / len(df)) * 100

kpis = {
    'Total Sales': df['Sales'].sum(),
    'Total Quantity Sold': df['Quantity_Sold'].sum(),
    'Total Profit': df['Profit'].sum(),
    'Average Order Size': avg_order_size,
    'Latest MoM Growth': monthly_sales['MoM Growth'].iloc[-1],
    'Latest YoY Growth': yearly_sales['YoY Growth'].iloc[-1],
    'Return Rate': return_rate
}

# Initialize Dash app
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("Superstore Sales Dashboard"),

    # Slicers
    html.Label("Product Category"),
    dcc.Dropdown(id='category-dropdown',
                 options=[{'label': cat, 'value': cat} for cat in df['Category'].unique()],
                 multi=True, value=df['Category'].unique()),

    html.Label("Region"),
    dcc.Dropdown(id='region-dropdown',
                 options=[{'label': reg, 'value': reg} for reg in df['Region'].unique()],
                 multi=True, value=df['Region'].unique()),

    html.Label("Sales Rep"),
    dcc.Dropdown(id='rep-dropdown',
                 options=[{'label': rep, 'value': rep} for rep in df['Sales_Rep_ID'].unique()],
                 multi=True, value=df['Sales_Rep_ID'].unique()),

    html.Label("Year"),
    dcc.Dropdown(id='year-dropdown',
                 options=[{'label': yr, 'value': yr} for yr in df['Year'].unique()],
                 multi=True, value=df['Year'].unique()),

    html.Label("Time of Day"),
    dcc.Dropdown(id='time-dropdown',
                 options=[{'label': t, 'value': t} for t in df['Time of Day'].unique()],
                 multi=True, value=df['Time of Day'].unique()),

    # Visualizations
    dcc.Graph(id='kpi-summary'),
    dcc.Graph(id='top-products'),
    dcc.Graph(id='top-regions'),
    dcc.Graph(id='top-customers'),
    dcc.Graph(id='sales-by-day'),
    dcc.Graph(id='sales-by-time'),
    dcc.Graph(id='sales-rep-performance'),
    dcc.Graph(id='monthly-trend'),
    dcc.Graph(id='hourly-sales'),
    dcc.Graph(id='category-pie'),
    dcc.Graph(id='sales-vs-returns'),
    dcc.Graph(id='profit-margin')
])

# Callback for updating graphs
@app.callback(
    [Output('kpi-summary', 'figure'),
     Output('top-products', 'figure'),
     Output('top-regions', 'figure'),
     Output('top-customers', 'figure'),
     Output('sales-by-day', 'figure'),
     Output('sales-by-time', 'figure'),
     Output('sales-rep-performance', 'figure'),
     Output('monthly-trend', 'figure'),
     Output('hourly-sales', 'figure'),
     Output('category-pie', 'figure'),
     Output('sales-vs-returns', 'figure'),
     Output('profit-margin', 'figure')],
    [Input('category-dropdown', 'value'),
     Input('region-dropdown', 'value'),
     Input('rep-dropdown', 'value'),
     Input('year-dropdown', 'value'),
     Input('time-dropdown', 'value')]
)
def update_graphs(categories, regions, reps, years, times):
    filtered_df = df[
        (df['Category'].isin(categories)) &
        (df['Region'].isin(regions)) &
        (df['Sales_Rep_ID'].isin(reps)) &
        (df['Year'].isin(years)) &
        (df['Time of Day'].isin(times))
    ]

    # KPI Summary
    filtered_kpis = {
        'Total Sales': filtered_df['Sales'].sum(),
        'Total Quantity Sold': filtered_df['Quantity_Sold'].sum(),
        'Total Profit': filtered_df['Profit'].sum(),
        'Average Order Size': filtered_df['Sales'].sum() / filtered_df['Quantity_Sold'].sum() if filtered_df['Quantity_Sold'].sum() > 0 else 0,
        'Return Rate': (filtered_df['Return_Flag'].sum() / len(filtered_df) * 100) if len(filtered_df) > 0 else 0
    }
    fig_kpi = go.Figure()
    for i, (key, value) in enumerate(filtered_kpis.items()):
        fig_kpi.add_trace(go.Indicator(
            mode="number",
            value=value,
            title={"text": key},
            domain={'x': [i/5, (i+1)/5], 'y': [0, 1]}
        ))
    fig_kpi.update_layout(title="KPIs Summary", height=300)

    # Top 2 Products
    top_products = filtered_df.groupby('Product Name')['Sales'].sum().nlargest(2).reset_index()
    fig_products = px.bar(top_products, x='Product Name', y='Sales', title='Top 2 Products by Sales')

    # Top 2 Regions
    top_regions = filtered_df.groupby('Region')['Sales'].sum().nlargest(2).reset_index()
    fig_regions = px.bar(top_regions, x='Region', y='Sales', title='Top 2 Regions by Sales')

    # Top 2 Customers
    top_customers = filtered_df.groupby('Customer Name')['Sales'].sum().nlargest(2).reset_index()
    fig_customers = px.bar(top_customers, x='Customer Name', y='Sales', title='Top 2 Customers by Sales')

    # Sales by Day of Week
    sales_by_day = filtered_df.groupby('Day of Week')['Sales'].sum().reindex(
        ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    ).reset_index()
    fig_day = px.bar(sales_by_day, x='Day of Week', y='Sales', title='Sales by Day of Week')

    # Sales by Time of Day
    sales_by_time = filtered_df.groupby('Time of Day')['Sales'].sum().reset_index()
    fig_time = px.bar(sales_by_time, x='Time of Day', y='Sales', title='Sales by Time of Day')

    # Sales Rep Performance
    sales_rep_perf = filtered_df.groupby('Sales_Rep_ID').agg({
        'Sales': 'sum',
        'Quantity_Sold': 'sum'
    }).reset_index().sort_values('Sales', ascending=False)
    fig_rep = px.bar(sales_rep_perf, x='Sales_Rep_ID', y='Sales',
                     title='Sales Rep Performance', hover_data=['Quantity_Sold'])

    # Monthly Sales Trend
    monthly_trend = filtered_df.groupby([filtered_df['Order Date'].dt.to_period('M')])['Sales'].sum().reset_index()
    monthly_trend['Order Date'] = monthly_trend['Order Date'].astype(str)
    fig_monthly = px.line(monthly_trend, x='Order Date', y='Sales', title='Monthly Sales Trend')

    # Hourly Sales
    hourly_sales = filtered_df.groupby('Time of Day')['Sales'].sum().reset_index()
    fig_hourly = px.bar(hourly_sales, x='Time of Day', y='Sales', title='Sales by Time of Day')

    # Sales by Category
    category_sales = filtered_df.groupby('Category')['Sales'].sum().reset_index()
    fig_category = px.pie(category_sales, names='Category', values='Sales', title='Sales by Product Category')

    # Sales vs Returns
    sales_returns = filtered_df.groupby('Region').agg({
        'Sales': 'sum',
        'Return_Flag': 'sum'
    }).reset_index()
    fig_sales_returns = px.bar(sales_returns, x='Region', y=['Sales', 'Return_Flag'],
                              barmode='group', title='Sales vs Returns')

    # Profit Margin
    profit_margin = filtered_df.groupby([filtered_df['Order Date'].dt.to_period('M')])['Profit'].sum().reset_index()
    sales_for_margin = filtered_df.groupby([filtered_df['Order Date'].dt.to_period('M')])['Sales'].sum().reset_index()
    profit_margin['Profit Margin'] = (profit_margin['Profit'] / sales_for_margin['Sales'] * 100) if sales_for_margin['Sales'].sum() > 0 else 0
    profit_margin['Order Date'] = profit_margin['Order Date'].astype(str)
    fig_profit = px.line(profit_margin, x='Order Date', y='Profit Margin', title='Profit Margin Over Time')

    return (fig_kpi, fig_products, fig_regions, fig_customers, fig_day, fig_time,
            fig_rep, fig_monthly, fig_hourly, fig_category, fig_sales_returns, fig_profit)

if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>

In [4]:
# Sales Growth Rate
monthly_sales = df.groupby([df['Order Date'].dt.to_period('M')])['Sales'].sum().reset_index()
monthly_sales['MoM Growth'] = monthly_sales['Sales'].pct_change() * 100
yearly_sales = df.groupby('Year')['Sales'].sum().reset_index()
yearly_sales['YoY Growth'] = yearly_sales['Sales'].pct_change() * 100

# Average Order Size
avg_order_size = df['Sales'].sum() / df['Quantity_Sold'].sum()

# Sales by Time of Day
sales_by_time = df.groupby('Time of Day')['Sales'].sum().reset_index()

# Return Rate
return_rate = (df['Return_Flag'].sum() / len(df)) * 100

# Print metrics
print(f"Average Order Size: ${avg_order_size:.2f}")
print(f"Return Rate: {return_rate:.2f}%")
print("\nMonthly Sales Growth (Sample):")
print(monthly_sales.tail())
print("\nYearly Sales Growth:")
print(yearly_sales)
print("\nSales by Time of Day:")
print(sales_by_time)

Average Order Size: $230.77
Return Rate: 5.04%

Monthly Sales Growth (Sample):
   Order Date        Sales  MoM Growth
43    2018-08   62837.8480   40.184500
44    2018-09   86152.8880   37.103499
45    2018-10   77448.1312  -10.103848
46    2018-11  117938.1550   52.280182
47    2018-12   83030.3888  -29.598366

Yearly Sales Growth:
   Year        Sales  YoY Growth
0  2015  479856.2081         NaN
1  2016  459436.0054   -4.255484
2  2017  600192.5500   30.636812
3  2018  722052.0192   20.303396

Sales by Time of Day:
  Time of Day        Sales
0   Afternoon  515767.2506
1     Evening  504893.3231
2     Morning  629274.3827
3       Night  611601.8263
